Install Tesseract

In [3]:
!sudo apt install tesseract-ocr


Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


Create Folders

In [4]:
!mkdir fonts output train

mkdir: cannot create directory ‘fonts’: File exists
mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘train’: File exists


Upload your fonts in fonts directory

Get the fonts Name and Assign that fonts


In [5]:
font_name = 'Mangal'

Mount google Drive

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Change Permission of Tesseract Folder

In [7]:
!chmod 755 -R /content/gdrive/MyDrive/tesseract_train/tesseract/src/training/tesstrain.sh

Extract the 'nep.lstm' from 'nep.traineddata'

In [8]:
!combine_tessdata -e /content/gdrive/MyDrive/tesseract_train/tesseract/tessdata/nep.traineddata nep.lstm

Extracting tessdata components from /content/gdrive/MyDrive/tesseract_train/tesseract/tessdata/nep.traineddata
Wrote nep.lstm
Version string:4.00.00alpha:nep:synth20170629:[1,48,0,1Ct3,3,16Mp3,3Lfys64Lfx96Lrx96Lfx512O1c1]
0:config:size=552, offset=192
17:lstm:size=11684995, offset=744
18:lstm-punc-dawg:size=3154, offset=11685739
19:lstm-word-dawg:size=690242, offset=11688893
20:lstm-number-dawg:size=162, offset=12379135
21:lstm-unicharset:size=7024, offset=12379297
22:lstm-recoder:size=994, offset=12386321
23:version:size=80, offset=12387315


**Generate Training Data**

Training data will be generates inside the 'train'folder

In [9]:
!rm -rf train/*
! /content/gdrive/MyDrive/tesseract_train/tesseract/src/training/tesstrain.sh --fonts_dir fonts \
  --fontlist '{font_name}' \
  --lang nep \
  --linedata_only \
  --langdata_dir /content/gdrive/MyDrive/tesseract_train/langdata_lstm_main \
  --tessdata_dir /content/gdrive/MyDrive/tesseract_train/tesseract/tessdata \
  --save_box_tiff \
  --maxpages 10 \
  --output_dir train



=== Starting training for language 'nep'
[Mon Jun 6 04:59:39 UTC 2022] /usr/bin/text2image --fonts_dir=fonts --ptsize 12 --font=Mangal --outputbase=/tmp/font_tmp.7FMH8zT59s/sample_text.txt --text=/tmp/font_tmp.7FMH8zT59s/sample_text.txt --fontconfig_tmpdir=/tmp/font_tmp.7FMH8zT59s
Stripped 1 unrenderable words
Rendered page 0 to file /tmp/font_tmp.7FMH8zT59s/sample_text.txt.tif

=== Phase I: Generating training images ===
Rendering using Mangal
[Mon Jun 6 04:59:41 UTC 2022] /usr/bin/text2image --fontconfig_tmpdir=/tmp/font_tmp.7FMH8zT59s --fonts_dir=fonts --strip_unrenderable_words --leading=32 --xsize=3600 --char_spacing=0.0 --exposure=0 --outputbase=/tmp/nep-2022-06-06.479/nep.Mangal.exp0 --max_pages=10 --font=Mangal --ptsize 12 --text=/content/gdrive/MyDrive/tesseract_train/langdata_lstm_main/nep/nep.training_text
Stripped 23 unrenderable words
Rendered page 0 to file /tmp/nep-2022-06-06.479/nep.Mangal.exp0.tif
Stripped 30 unrenderable words
Rendered page 1 to file /tmp/nep-2022-06

Finetune 

In [10]:
%%shell

rm -rf output/* 
OMP_THREAD_LIMIT=16 lstmtraining \
	--continue_from nep.lstm \
	--model_output output/font_name \
	--traineddata /content/gdrive/MyDrive/tesseract_train/tesseract/tessdata/nep.traineddata \
	--train_listfile train/nep.training_files.txt \
	--max_iterations 200

Loaded file nep.lstm, unpacking...
Continuing from nep.lstm
Loaded 390/390 pages (1-390) of document train/nep.Mangal.exp0.lstmf
2 Percent improvement time=46, best error was 100 @ 0
At iteration 46/100/100, Mean rms=0.481%, delta=0.392%, char train=1.149%, word train=3.775%, skip ratio=0%,  New best char error = 1.149 Transitioned to stage 1 wrote best model:output/font_name1.149_46.checkpoint wrote checkpoint.

2 Percent improvement time=85, best error was 100 @ 0
At iteration 85/200/200, Mean rms=0.45%, delta=0.322%, char train=0.953%, word train=3.271%, skip ratio=0%,  New best char error = 0.953 wrote best model:output/font_name0.953_85.checkpoint wrote checkpoint.

Finished! Error rate = 0.953


##**Get Trained Data**
This command will create trained data from fontname.checkpoint
This will be inside Output folder

In [11]:
%%shell

lstmtraining --stop_training \
	--continue_from output/font_name_checkpoint \
	--traineddata /content/gdrive/MyDrive/tesseract_train/tesseract/tessdata/nep.traineddata \
	--model_output output/font_name.traineddata

Loaded file output/font_name_checkpoint, unpacking...
